# Introdução

O dataset foi obtido no [Kaagle](https://www.kaggle.com/jmessiasalves/ufpi-ncad-iot-attacks) e é sobre ataques em redes IoT. 

## Questões interessantes para serem analisadas no dataset

 - Analisar como os ataques funcionam.
 - Entender o comportamento.
 - Prever ataques.
 - Analisar possíveis correlações não exploradas.
 
### Equipe
 - Igor Melo
 - Jarélio Filho
 - Leyberson Assunção
 - Gustavo Mota
 - Eduardo Linhares

# 1 - Conhecendo os dados
# 1.1 - Leitura dos dados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno
import os

%matplotlib inline

In [2]:
if 'UFPI-NCAD-IoT-Attacks-all-v1.csv' not in os.listdir():
    !unzip 578734_1047033_bundle_archive.zip

'unzip' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


In [ ]:
df_description = pd.read_csv('UFPI-NCAD-IoT-Attacks-all-v1-Description.txt', sep='|')
# pd.read_csv('UFPI-NCAD-IoT-Attacks-all-v1.csv', sep=';')

# There is two actual ways to read the dataset
path = 'UFPI-NCAD-IoT-Attacks-all-v1.csv'
df_target = pd.read_csv(path, sep=';', low_memory=False)
#df_target = pd.read_csv(path,sep=';', engine='python')

# 1.2 - Análise Inicial
Aqui observamos pela primeira vez os dados e identificamos:

- O que significam os dados armazenados
- Os tipos dos dados
- Volume de dados NaNs
- Problema no dataset

In [ ]:
df_description

In [ ]:
df_target.info()

In [ ]:
df_target

In [ ]:
miss = missingno.bar(df_target)

In [ ]:
df_target.label.value_counts()

# 1.3 - Resolvendo o problema de transposição de colunas

Identificamos que o problema é que são dois datasets juntos em que o segundo não possui duas das colunas do primeiro. Há um cabeçalho em uma linha específica. Dividimos os dois, concatenamos e refazemos a última análise.

Na próxima célula, percebemos uma linha que contém um novo cabeçalho. Esta é a linha que começa o segundo dataset. Na duas células seguintes, perceberá

&nbsp; &nbsp; 1. Um shift dos valores das colunas.

&nbsp; &nbsp; 2. A contagem de valores na coluna ipv6.src mostrando a diferença entre os valores existentes. Além disso há uma mudança no padrão dos elementos nessa coluna. Um endereço ipv6 não tem as caracteristicas dos apresentados no final do output. Na verdade, os valores ao final são característicos de outro tipo de endereço.

In [ ]:
df_target[df_target['frame.time_delta'] == 'frame.time_delta']

In [ ]:
df_target[['frame.time_delta', 'frame.time_epoch', 'frame.time_relative']]

In [ ]:
df_target['ipv6.src'].value_counts()

## 1.3.1 Como nossa correção funciona
Primeiro, gravamos o primeiro dataset separadamente em um arquivo. Em seguida, repetimos o processo para o segundo dataset. Por fim, fazemos a leitura dos dois e concatenamos. 

As duas células abaixo são respectivamente a separação e escrita dos datasets e a leitura e concatenação. Caso você já tenha os datasets separados, pode executar apenas a célula seguinte.


In [ ]:
# Spliting and Writing
df_target.iloc[:670052].to_csv('first_dataset.csv', index = False)
df_target.iloc[670052:].drop(columns=['mqtt.willtopic', 'label'], axis=0).to_csv('second_dataset.csv', header=False, index=False)
del df_target

In [ ]:
# Reading and concatenating splited data
df = pd.read_csv('first_dataset.csv', low_memory=False)
df_= pd.read_csv('second_dataset.csv', low_memory=False)

df_concat = pd.concat([df, df_])

Nas três células abaixo certificamos a correção pela atualização do gráfico de missing values para verificar os valores mais apurados, contamos novamente os ataques por labels e observamos o dataset corrigido.

In [ ]:
missingno.bar(df_concat)

In [ ]:
df_concat["label"].value_counts()

In [ ]:
df_concat

# 2 - De volta para a análise de dados
Aqui analisamos as correlações, exploramos a proporção de valores NaN no data set para futuras implicações e possíveis reparações de dados e algumas análises básicas.

# 2.1 Correlações

Nas três células abaixo, geramos e exibimos as matrizes de correlação. Porém, as matrizes valem para correlações lineares.

In [ ]:
correOne = first_dataset.corr()
correTwo = second_dataset.corr()

In [ ]:
correOne

In [ ]:
corre_final = df_concat.corr()
corre_final

## Conclusões das correlações identificadas: 
- A matriz de correlações do dataset final ficou menor. 
- Vamos identificar posteriormente a proporção de NaNs em relação à quantidade de valores válidos. Os valores de NaNs importam para a identificação de um ataque, porque caracteristicamente nem todas as informações podem sequer existir.

# 2.1.1 Matrix de correlações

As correlações anteriores são lineraes, mas provavelmente correlações quadráticas, por exemplo, não seriam identificadas. Abaixo há um heatmap e uma matrix de correlações que nos mostra, scartterplots para identificar mais correlações possíveis.

In [ ]:
corr = df.corr()
corr.dropna(how='all', inplace=True)
corr.dropna(how='all', axis=1, inplace=True)
corr.style.background_gradient(cmap='Blues')

In [ ]:
corr = df.corr()
corr.dropna(how='all', inplace=True)
corr.dropna(how='all', axis=1, inplace=True)
corr.style.background_gradient(cmap='Blues')

In [ ]:
sns.pairplot(df.drop(columns = ['mqtt.willmsg']))

Conclusões da matrix: 

## 2.1.2 Análises dos NaN

A contagem de NaNs é importante para a análise dos ataques. Nas duas céulas abaixo há uma função para contagem absoluta de NaNs e a contagem para o primeiro, o segundo e o dataset final.

In [ ]:
def nan_counter(dataframe): # Counts the values of NaN
  nan_counts = {}
  for column in dataframe.columns:
    nan_counts[column] = dict(dataframe[column].isna().value_counts())
  return nan_counts

nan_counter(df_concat)

In [ ]:
nan_first = nan_counter(first_dataset)
nan_second = nan_counter(second_dataset)

nan_final = nan_counter(df_concat)

O seguinte gráfico de barras 

In [ ]:
# Barplot NaN

def plot_doublebar(dicte):
  grupos = len(dicte)
  
  false = [value['False'] for value in dicte.values()]
  true = [value['True'] for value in dicte.values()]
  fig, ax = plt.subplots()
  indice = np.arange(grupos)
  bar_larg = 0.4
  transp = 0.7
  plt.bar(indice, false, bar_larg, alpha=transp, label='Value')
  plt.bar(indice + bar_larg, true, bar_larg, alpha=transp, label='NaN')

  plt.xlabel('Type') 
  plt.ylabel('Count') 
  plt.title('Values and NaN count per type') 
  plt.xticks(indice + bar_larg, (list(dicte.keys())), rotation=80)
  plt.legend() 
  #plt.tight_layout() 
  plt.show()

plot_doublebar(nan_final)

Acima vemos novamente a proporção de NaNs comparado com a de valores existentes. 

mqtt.msg e mqtt.topic podem ter valor nenhum e ser um indício de ataques como: bruteforce e mitm, que somente se utilizam de mensagens de controle e conexão na rede, não contendo o conteúdo que um dispositivo padrão de uma rede IoT prôve ao broker. podemos fazer uma classificação nas mensagens de mqtt.msg para classfificá-la em ataques e não ataques fazendo análise das substrings. 

Os valores NaN podem ser causados por ataque. Portanto, podemos relacionar com mqtt.topic e trabalhar em uma predição como veremos mais adiante.

In [ ]:
# Padrão esquisito
dstport = np.array(df_concat['tcp.dstport'].isna())
srcport = np.array(df_concat['tcp.srcport'].isna())

comparisson = np.where(dstport == srcport, dstport, srcport)

true_corr = np.count_nonzero(comparisson == True)
false_corr = np.count_nonzero(comparisson == False)

# False and true counts of dstport and srcport
tc_dst = np.count_nonzero(dstport == True)
tc_src = np.count_nonzero(srcport == True)
fc_dst = np.count_nonzero(dstport == False)
fc_src = np.count_nonzero(srcport == False)

print('Comparisson true:', true_corr,
      '\nComparisson false:', false_corr,
      '\ndstport true:', tc_dst,
      '\ndstport false:', fc_dst,
      '\nsrcport true:', tc_src,
      '\nsrcport false:', fc_src,
      '\nsize dstport:', len(dstport),
      '\nsize srcport:', len(srcport),
      '\nproportion comparisson both:', true_corr/false_corr,
      '\nproportion dstport both:', tc_dst/fc_dst,
      '\nproportion srcport both:', tc_src/fc_src,
      '\nproportion comparisson true:', true_corr/len(comparisson),
      '\nproportion comparisson false:', false_corr/len(comparisson),
      '\npropotion dstport true:', tc_dst/len(dstport),
      '\npropotion dstport false:', fc_dst/len(dstport),
      '\npropotion srcport true:', tc_src/len(srcport),
      '\npropotion srcport false:', fc_src/len(srcport)
)

As colunas dstport e srcport, respectivamente, corresponde a porta de destino do pacote e a porta de saída do pacote. 

Existe uma forte correlação entre a porta de destino e a porta de saída como observado na matrix de correlações: -0.956167. Isso se deve ao fato de que comunicação em rede é feita de maneira padronizada com a mesma porta baixa sendo usada pelo broker e portas altas sendo usadas pelos dispostivos.

Nós encontramos 62507 tuplas com valores NaN, isto se iguala à quantidade de valores NaN nas colunas srcport e dstport. 

Também percebemos que existem poucas tuplas com essas mesmas colunas NaN. Mas também suficientes para usarmos um algoritmo treinado e predizermos os valores faltantes.

In [ ]:
# Adaptar para mqtt.msg mqtt.topic
dstport = np.array(df_concat['mqtt.msg'].isna())
srcport = np.array(df_concat['mqtt.topic'].isna())

comparisson = np.where(dstport == srcport, dstport, srcport)

true_corr = np.count_nonzero(comparisson == True)
false_corr = np.count_nonzero(comparisson == False)

# False and true counts of dstport and srcport
tc_dst = np.count_nonzero(dstport == True)
tc_src = np.count_nonzero(srcport == True)
fc_dst = np.count_nonzero(dstport == False)
fc_src = np.count_nonzero(srcport == False)

print('Comparisson true:', true_corr,
      '\nComparisson false:', false_corr,
      '\ndstport true:', tc_dst,
      '\ndstport false:', fc_dst,
      '\nsrcport true:', tc_src,
      '\nsrcport false:', fc_src,
      '\nsize dstport:', len(dstport),
      '\nsize srcport:', len(srcport),
      '\nproportion comparisson both:', true_corr/false_corr,
      '\nproportion dstport both:', tc_dst/fc_dst,
      '\nproportion srcport both:', tc_src/fc_src,
      '\nproportion comparisson true:', true_corr/len(comparisson),
      '\nproportion comparisson false:', false_corr/len(comparisson),
      '\npropotion dstport true:', tc_dst/len(dstport),
      '\npropotion dstport false:', fc_dst/len(dstport),
      '\npropotion srcport true:', tc_src/len(srcport),
      '\npropotion srcport false:', fc_src/len(srcport)
)

Vamos descobrir a proporção de ataques em dstport e srcport para conferir se corresponde à proporção de labels e identificar se o experimento segue os padrões de ataques ou está usando maneiras novas de promover um ataque # adicionar link sobre os ataques costumam ser em portas baixas de cliente, vamos verificar as dstport.

In [ ]:
# identifica características para melhor avaliar a credibilidade do expe como
# a diversidade de portas dst e src usadas e como isso afeta

dst_counts = df_concat['tcp.dstport'].value_counts()
min = df_concat['tcp.dstport'].min()
max = df_concat['tcp.dstport'].max()
amp = max - min

mean = df_concat['tcp.dstport'].mean()
std = df_concat['tcp.dstport'].std()
mode = df_concat['tcp.dstport'].mode()
median = df_concat['tcp.dstport'].median()

src_counts = df_concat['tcp.srcport'].value_counts()
src_min = df_concat['tcp.srcport'].min()
src_max = df_concat['tcp.srcport'].max()
src_amp = src_max - src_min

src_mean = df_concat['tcp.srcport'].mean()
src_std = df_concat['tcp.srcport'].std()
src_mode = df_concat['tcp.srcport'].mode()
src_median = df_concat['tcp.srcport'].median()

cv_srcport = src_std/src_mean
cv_dstport = std/mean

dstcounts_df = dst_counts.rename_axis('ports').reset_index(name='counts')

tmp = dstcounts_df['counts'] == 1 # tmp = dstcounts_df['counts'].value_counts() == 1
unique = tmp.value_counts()[True] # número de portas usadas só uma vez

print('Max dstport value:', max, '. Min dstport value:', min,". Amplitude:",amp,". Mode:",mode,
      "\nMean of dstport values:", "{:.2f}".format(mean),". Standart deviation of dstport values:", "{:.2f}".format(std),". Median: ", median,

      '\n\nMax srcport value:', "{:.2f}".format(src_max), '. Min srcport value:', "{:.2f}".format(src_min),". Amplitude:",src_amp,". Mode:", src_mode,
      "\nMean of srcport values:", "{:.2f}".format(src_mean),". Standart deviation of srcport values:", "{:.2f}".format(src_std),". Median: ", src_median,

      "\n\nCoefficient of Variation from dstport:", "{:.2f}".format(cv_dstport)," from srcport:", "{:.2f}".format(cv_srcport),
      "\nNumber of ports used just one time:",unique)

Tem muita porta diferente, o que indica uma variedade muito grande

Observe que como estamos falando de portas e são praticamente discretas ao contrário de números como preços, a média e o desvio padrão apenas são úteis para calcular os coeficientes

Poucas portas foram usadas uma só vez, é claro

O que significa que o experimento tem alguma fidelidade nesse quesito pois uma rede IoT real um número certamente finito de portas sob ataque terá várias portas de cliente usadas mais de uma vez, pelo princípio da casa dos pombos

## 2.1.3 Portas Mais e Menos Usadas

In [ ]:
# Piechart portas mais e menos usadas e qtd

dstcounts_df = dst_counts.rename_axis('ports').reset_index(name='counts')

piechart_ports = np.unique(dstcounts_df['counts'].value_counts().to_numpy())

plt.figure(0)
ports0 = dstcounts_df['ports'][:10]
counts0 = dstcounts_df['counts'][:10]

fig0 = plt.figure(figsize =(10, 7)) 
plt.pie(counts0, labels = ports0) 

plt.figure(1)
ports1 = dstcounts_df['ports'][dstcounts_df.shape[0]-10:]
counts1 = dstcounts_df['counts'][dstcounts_df.shape[0]-10:]
  
# Creating plot 
fig1 = plt.figure(figsize =(10, 7)) 
plt.pie(counts1, labels = ports1)

plt.show() 

Acima temos dois gráficos mostrando a proporção da quantidade de vezes que uma porta foi usada. O gráfico acima mostra as mais usadas. A porta 1883 foi usada muitas vezes, pois é usada pelo broker para receber os dados da rede. 

O gráfico abaixo mostra as menos usadas. É importante notar que todas essas portas foram usadas somente uma vez, o que é improvável para um dispositivo padrão pois ele utiliza a mesma porta para continuamente mandar seus dados em tempo real. É possível que a quantidade de vezes que uma porta alta foi utilizada possa determinar comportamento anormal na rede.

# 2.2 Continuidade das análises

# Chequando correlações para determinar ataques DoS

The following analysis uses only first_dataset.csv generated by subdataset notebook .

In [ ]:
df = pd.read_csv('first_dataset.csv', low_memory=False, sep=',')

## Analisando a correlação entre o tamanho do pacote e o tempo de resposta

In [ ]:
#There's exactly one value in this column that goes beyond a  float64 which is weird but is ignored, probably it's the datatype
frame_delta = pd.to_numeric(df['frame.time_delta'], errors='coerce') 
packet_lenght = df['ipv6.plen']

correlation = frame_delta.corr(packet_lenght)
print(correlation)

### Não existe correlação, isso se deve a fragmentação de pacotes, de acordo com a RFC8200 (https://tools.ietf.org/html/rfc8200) em jullho 2017, que padroniza o ipv6, o tamanho máximo de um pacote ipv6 é igual a MTU (maximum transmiter unit) do link de conexão, de acordo com a mesma, o minímo que uma rede ipv6 necessita de vazão de dados em um link de conexão é de 1280 bytes. O único caso em que um pacote não é fragmentado é quando ele excede o tamanho da MTU, nesse caso ele é transmitido por mais de uma MTU simultâneamente.

### Aqui checamos de algum pacote pode não ser fragmentado

In [ ]:
df.loc[df['ipv6.plen'] >= 1280*8]

## Analisar correlação entre tamanho da mensagem e tempo de resposta

In [ ]:
frame_delta = pd.to_numeric(df['frame.time_delta'], errors='coerce') 
mqtt_lenght = pd.to_numeric(df['mqtt.len'], errors='coerce')

correlation = mqtt_lenght.corr(packet_lenght)
print(correlation)

### Essa correlação existe e é fácil de observar, mensagens maiores demoram mais tempo para serem processadas pelo servidor, entretanto a partir disso é possível buscar uma proporção com o tamanho da mensagem ou com o tempo de resposta e identificar se ataques de negação de serviço atingem tempos absurdos de tempo de resposta ou valores extremamente altos no tamanho da mensagem

### Obs: Os valores do tempo de resposta são extremamente precisos em algumas tuplas, então estas foram excluidas dessas analises iniciais pois requerem um tratamento especial de dados.

### Tuplas excluídas:

In [ ]:
df['frame.time_delta'] = pd.to_numeric(df['frame.time_delta'], errors='coerce')
df[df['frame.time_delta'].isnull()]

In [ ]:
#670052 100 
#95642 x
print(str((95642*100)/670052) + "% of the tuples were excluded in the first correlation")

In [ ]:
df['mqtt.len'] = pd.to_numeric(df['mqtt.len'], errors='coerce')
df[df['mqtt.len'].isnull()].loc[df['label'] != 'normal']

In [ ]:
#670052 100 
#322215  x
print(str((105283*100)/670052) + "% of the tuples were excluded in the second correlation this may or may not overlap with the loss from the first correlation which is still present")

## 2.2.1 Categorizando ataques MITM

### Em uma rede normal o relacionamento padrão entre endereço IP e MAC é de 1 para 1, ou seja, um IP aponta para um MAC e um MAC é apontado por somente um IP. Entretanto, em casos de ataque MITM, o atacante se passa por um dos nós da rede, para isso ele pode tanto copiar o MAC do dispostivo que ele deseja "escutar" ou mudar a tabela ARP da rede para resolver seu IP para o MAC do alvo.

### Aqui estamos agrupando os valores em questão para fazer uma breve análise sobre essa hipótese

In [11]:
df2.groupby(['eth.src', 'ipv6.src', 'label']).agg({'frame.time_delta': 'count'})

frame.time_delta
eth.src           ipv6.src                             label                        
08:00:27:e4:e6:83 fd9e:6c51:2336::1                    mitm                     2144
                  fe80::32b5:c2ff:fe4c:238a            mitm                     1488
                  fe80::7a78:397a:82c6:761b            mitm                     1560
2c:f4:32:19:85:be fd9e:6c51:2336:0:2ef4:32ff:fe19:85be normal                  22975
30:b5:c2:4c:23:8a fd9e:6c51:2336::1                    mitm                     6226
                                                       normal                   1105
                  fe80::32b5:c2ff:fe4c:238a            mitm                     2704
                                                       normal                    116
60:01:94:0e:87:7a fd9e:6c51:2336:0:6201:94ff:fe0e:877a normal                  22958
68:a3:c4:6e:50:12 fd9e:6c51:2336:0:717b:e02b:9c00:d43a enumeration              4276
                                                       normal                 140849
68:c6:3a:99:40:54 fd9e:6c51:2336:0:6ac6:3aff:fe99:4054 normal                  22572
80:86:f2:f9:f9:4d fd9e:6c51:2336:0:195a:4614:25ac:dd31 mitm                      240
                  fd9e:6c51:2336:0:8477:491b:87c1:bd39 mitm                      704
                  fd9e:6c51:2336:0:9873:f160:5832:6509 enumeration              4694
                  fe80::4056:3a9a:e497:8e7d            mitm                      684
                  fe80::7a78:397a:82c6:761b            mitm                    39062
cc:50:e3:1c:2a:5b fd9e:6c51:2336:0:ce50:e3ff:fe1c:2a5b normal                  21875
                  fe80::ce50:e3ff:fe1c:2a5b            normal                    524
cc:50:e3:1c:32:ec fd9e:6c51:2336:0:ce50:e3ff:fe1c:32ec mitm                      432
                                                       normal                  22975
                  fe80::ce50:e3ff:fe1c:32ec            mitm                      816
                                                       normal                      1
cc:50:e3:55:cd:91 fd9e:6c51:2336:0:ce50:e3ff:fe55:cd91 normal                  24613

In [13]:
_11.reset_index()

,eth.src,ipv6.src,label,frame.time_delta
0,08:00:27:e4:e6:83,fd9e:6c51:2336::1,mitm,2144
1,08:00:27:e4:e6:83,fe80::32b5:c2ff:fe4c:238a,mitm,1488
2,08:00:27:e4:e6:83,fe80::7a78:397a:82c6:761b,mitm,1560
3,2c:f4:32:19:85:be,fd9e:6c51:2336:0:2ef4:32ff:fe19:85be,normal,22975
4,30:b5:c2:4c:23:8a,fd9e:6c51:2336::1,mitm,6226
5,30:b5:c2:4c:23:8a,fd9e:6c51:2336::1,normal,1105
6,30:b5:c2:4c:23:8a,fe80::32b5:c2ff:fe4c:238a,mitm,2704
7,30:b5:c2:4c:23:8a,fe80::32b5:c2ff:fe4c:238a,normal,116
8,60:01:94:0e:87:7a,fd9e:6c51:2336:0:6201:94ff:fe0e:877a,normal,22958
9,68:a3:c4:6e:50:12,fd9e:6c51:2336:0:717b:e02b:9c00:d43a,enumeration,4276


### Como pudemos observar, realmente o label MITM está relacionado com uma relação não padrão entre IP e MAC. Temos tanto mais de um IP resolvendo para o mesmo MAC quanto um IP com diferentes MACs ao longo da captura de pacotes da rede. Em todos os casos observados no gráfico, essa anomalia está associada à um ataque MITM. Então podemos investigar a detecção de tal ataque baseada nessa hipótese, também podemos treinar um algoritmo que escuta a rede durante determinado período e então com base nos pacotes escutados identifica o nó do invasor.